In [ ]:
# What are we watching tonight?
# Which year(s) had the most highly rated shows?

In [ ]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from scipy.stats import linregress

from config.py import api_key

In [ ]:
url=f"https://api.watchmode.com/v1/list-titles/?apiKey={api_key}"

# For loop through for your individual task

query_url=f"{url} +"

In [ ]:
# Pull data and convert to json
data=requests.get(query_url)
data_json=data.json()

In [ ]:
# Create dataframe
data_df=pd.DataFrame(data_json)
data_df.head()

In [ ]:
# Clean dataframe
clean_df=data_df.dropna()
clean_df.head()